In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a05_unidades_de_apoyo as ua
from proyeccion_rdr.produccion.a04_ambulatorio import (
    leer_consultas_medicas
)


pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

2025-05-30 13:26:25.222 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr


In [5]:
# Define las rutas de los archivos
ruta_raw = "../data/raw/"

# Define las rutas de la base de CAE y GRD
ruta_cae = ruta_raw + "6_ambulatorio/df_procesada_consultas.csv"
ruta_grd = ruta_raw + "7_egresos_grd/df_procesada_grd.csv"
ruta_urgencias = ruta_raw + "8_urgencias/df_procesada_urgencias.csv"

# Define las rutas de los archivos de apoyo diagnostico
ruta_apoyo = ruta_raw + "9_unidades_de_apoyo/"

ruta_laboratorio = ruta_apoyo + "df_procesada_laboratorio_clinico.csv"
ruta_farmacia = ruta_farmacia_amb = ruta_apoyo + "df_procesada_farmacia.csv"
ruta_imagenologia = ruta_apoyo + "df_procesada_imagenologia.csv"
ruta_umt = ruta_apoyo + "df_procesada_umt.csv"

In [6]:
# Lee CAE y GRD
consultas_medicas = leer_consultas_medicas(ruta_cae)
df_grd = ua.leer_grd(ruta_grd)

C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:90: DtypeWarning: Columns (10,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})
C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a05_unidades_de_apoyo.py:11: DtypeWarning: Columns (47,48,49,50,51,52,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})
